# TP-6 Texture synthesis

### Student information
Student: Balthazar Neveu

🔗 [Github](https://github.com/balthazarneveu/MVA24-delire) | [Online page for this class lab session](https://balthazarneveu.github.io/MVA24-delire)

⭐ [Online HTML version of this notebook](https://balthazarneveu.github.io/MVA24-delire/TP_6/tp_6.html)